In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'  # default is ‘last_expr’

%load_ext autoreload
%autoreload 2

In [3]:
import sys
sys.path.append('/lib/ai4eutils')

In [4]:
import json
import os

from tqdm import tqdm
import numpy as np
import rasterio
from PIL import Image

from geospatial.visualization.imagery_visualizer import ImageryVisualizer

# Append DEM as a channel to the Landsat scenes and tiles

Experiments show that adding the elevation data as an additional channel helped model performance. In this notebook, we append the DEM layer to the Landsat 8 layers, because the Land Cover Mapping tool requires that all input data be stacked together.

The DEM layer is the SRTM data.

The Landsat scenes are of dtype float64/float32. The DEM data is in int16, but will be normalized and saved as a float when appended.

In [ ]:
# Where the blob storage container is mounted. All paths below are specified relative to this
container_mnt = '/wcs/mnt/wcs-orinoquia'  

datasets = [
    {
        'input_dir': 'images_sr_median/2013_2014',
        'output_dir': 'images_sr_median/2013_2014_dem'
    },
    {
        'input_dir': 'images_sr_median/2019_202004',
        'output_dir': 'images_sr_median/2019_202004_dem'
    },
    {
        # we previously already tiled the historical scenes for training; append DEM to them too so that our training
        # dataloader can be reused for the interactive tool
        'input_dir': 'tiles/full_sr_median_2013_2014/tiles',
        'output_dir': 'tiles/full_sr_median_2013_2014/tiles_dem',
    }
]

srtm_path = 'images_srtm/wcs_orinoquia_srtm.tif'


elevation_standardization_params = {
    # from calculations done in GEE
    'mean': 399.78,
    'std_dev': 714.78
}

In [ ]:
elevation_reader = rasterio.open(os.path.join(container_mnt, srtm_path))

for d in datasets:
    input_dir = os.path.join(container_mnt, d['input_dir'])
    output_dir = os.path.join(container_mnt, d['output_dir'])
    
    os.makedirs(output_dir, exist_ok=True)
    
    for raster_path in tqdm(os.listdir(input_dir)):
        if not raster_path.endswith('.tif'):
            continue
        
        output_path = os.path.join(output_dir, raster_path)
        if os.path.exists(output_path):
            continue
            
        print(input_dir, raster_path)
        
        raster_reader = rasterio.open(os.path.join(input_dir, raster_path))
        
        # get the window to crop the corresponding scene from the big DEM image
        x_min = raster_reader.bounds.left
        y_min = raster_reader.bounds.top  # origin is left, top
        
        # getting the pixel array indices corresponding to points in georeferenced space
        row, col = elevation_reader.index(x_min, y_min)

        # tile wcs_orinoquia_sr_median_2013_2014-0000000000-0000007424_-72.425_7.671.tif
        # top left corner looks up to a negative row index. Clipping to 0 seems to be okay visually
        row = max(0, row)
        col = max(0, col)
        
        row_end = row + raster_reader.height  
        col_end = col + raster_reader.width 
        
        w = rasterio.windows.Window.from_slices((row, row_end), (col, col_end))
        
        dem_layer: np.ndarray = elevation_reader.read(1, window=w)  # only 1 band
        # standardize the DEM layer
        dem_layer = (dem_layer - elevation_standardization_params['mean']) / elevation_standardization_params['std_dev']
        dem_layer = np.expand_dims(dem_layer, axis=0)
        
        # concatenate the dem_layer to the Landsat channels
        
        landsat_channels = raster_reader.read()  # read all Landsat channels into a numpy array
        
        # match extent
        landsat_width, landsat_height = landsat_channels.shape[1], landsat_channels.shape[2]
        dem_width, dem_height = dem_layer.shape[1], dem_layer.shape[2]
        min_width = min(landsat_width, dem_width)
        min_height = min(landsat_height, dem_height)
        
        landsat_channels = landsat_channels[:, :min_width, :min_height]
        dem_layer = dem_layer[:, :min_width, :min_height]
        
        stacked = np.concatenate([landsat_channels, dem_layer]).astype(np.float32)
        assert stacked.shape[0] == 11
        
        profile = raster_reader.profile
        # driver='COG' does not yet work
        profile.update(count = raster_reader.count + 1, dtype=rasterio.float32, compress='lzw', driver='GTiff')  
        
        with rasterio.open(output_path, 'w', **profile) as dst:
            dst.write(stacked)
        
        raster_reader.close()

elevation_reader.close()

# Create a .vrt for the scenes and tiles

So that the interactive tool can load each as a dataset.

```
gdalbuildvrt wcs_orinoquia_sr_median_2013_2014_dem.vrt ./wcs_orinoquia_sr_median_2013_2014-*.tif
```

Create the basemap:
```
gdal_translate wcs_orinoquia_sr_median_2013_2014_dem.vrt wcs_orinoquia_sr_median_2013_2014_rgb_exp.vrt -of vrt -ot Byte -scale_1 0 3000 -scale_2 0 3000 -scale_3 0 3000 -exponent 0.7 -co PHOTOMETRIC=RGB -b 4 -b 3 -b 2 -colorinterp red,green,blue 
```

The scaling and exponent scaling values are chosen for the Landsat 8 image to look good.


```
gdal2tiles.py --processes=6 -z 8-13 wcs_orinoquia_sr_median_2013_2014_rgb_exp.vrt basemap_exp07/
```

In [ ]:
reader = rasterio.open('/wcs/mnt/wcs-orinoquia/images_sr_median/2013_2014_dem/wcs_orinoquia_sr_median_2013_2014_rgb_exp.vrt')

In [ ]:
reader.shape

In [ ]:
w = rasterio.windows.Window.from_slices((10000, 10000 + 600), (10000, 10000 + 600))

In [ ]:
array = reader.read([1, 2, 3], window=w)

In [ ]:
array.shape
array.min()
array.max()

In [ ]:
array = array.transpose((1, 2, 0))

In [ ]:
ImageryVisualizer.show_patch(reader, 
                             window=w,
                             bands=[1, 2, 3], 
                             size=(600, 600),
                             band_min=0,
                             band_max=256)
# (col_off x, row_off y, width delta_x, height delta_y)

# Color map to interactive tool format

In [23]:
from geospatial.visualization.raster_label_visualizer import RasterLabelVisualizer

In [24]:
label_vis = RasterLabelVisualizer('/wcs/pycharm/constants/class_lists/wcs_coarse_label_map.json')

In [25]:
classes = label_vis.get_tool_colormap()